In [1]:
import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy as sa
import os
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.types import Integer, Float, String, DateTime
from transformation_scripts import fact_transformation, dates_transformation

In [2]:
warehouse_url = f"mysql+pymysql://{os.environ['TEST_DB_USERNAME']}:{os.environ['TEST_DB_PASSWORD']}@{os.environ['TEST_DB_HOSTNAME']}/lung_cancer_air_pollution_warehouse_test"
warehouse_engine = create_engine(warehouse_url)


In [3]:
try:
    with warehouse_engine.connect() as connection:
        print("Connection to MySQL database successful!")
except Exception as e:
    print(f"Error: {e}")

Connection to MySQL database successful!


In [4]:
type(fact_transformation())

Column BaP not present already


pandas.core.frame.DataFrame

In [5]:
df = fact_transformation()

Column BaP not present already


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187482 entries, 0 to 187481
Data columns (total 19 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   patient_id          187482 non-null  int64  
 1   patient_age         187482 non-null  int32  
 2   patient_gender      187482 non-null  object 
 3   country             187482 non-null  object 
 4   diagnosis_date      187482 non-null  object 
 5   end_treatment_date  187482 non-null  object 
 6   cancer_stage        187482 non-null  object 
 7   hypertension        187482 non-null  int64  
 8   asthma              187482 non-null  int64  
 9   cirrhosis           187482 non-null  int64  
 10  treatment_type      187482 non-null  object 
 11  survived            187482 non-null  int64  
 12  pm25_index          155926 non-null  float64
 13  pm10_index          169693 non-null  float64
 14  no2_index           163347 non-null  float64
 15  so2_index           157744 non-nul

In [7]:
with warehouse_engine.begin() as connection:
    diagnosis_date_ids = connection.execute(sa.text("SELECT date_id, date FROM dim_dates"))
    end_treament_ids = connection.execute(sa.text("SELECT date_id, date FROM dim_dates"))
    treament_type_ids = connection.execute(sa.text("SELECT * FROM dim_treatment_types"))
    cancer_stage_ids = connection.execute(sa.text("SELECT * FROM dim_cancer_stages"))
    air_quality_ids = connection.execute(sa.text("SELECT air_quality_id, air_quality FROM dim_air_qualities"))
    country_codes = connection.execute(sa.text("SELECT country_code, country_name FROM dim_countries"))

In [8]:
dd = diagnosis_date_ids.all()
dd_df = pd.DataFrame(dd, columns=diagnosis_date_ids.keys())

etd = end_treament_ids.all()
etd_df = pd.DataFrame(etd, columns=end_treament_ids.keys())

tt = treament_type_ids.all()
tt_df = pd.DataFrame(tt, columns=treament_type_ids.keys())

cs = cancer_stage_ids.all()
cs_df = pd.DataFrame(cs, columns=cancer_stage_ids.keys())

aq = air_quality_ids.all()
aq_df = pd.DataFrame(aq, columns=air_quality_ids.keys())

cc = country_codes.all()
cc_df = pd.DataFrame(cc, columns=country_codes.keys())

In [9]:
cs_df

,cancer_stage_id,cancer_stage
0,s1,Stage I
1,s2,Stage II
2,s3,Stage III
3,s4,Stage IV


In [11]:
result_df = df.merge(dd_df, left_on='diagnosis_date', right_on='date', how='inner') \
               .merge(etd_df, left_on='end_treatment_date', right_on='date', how='inner') \
               .merge(tt_df, left_on='treatment_type', right_on='treatment', how='inner') \
               .merge(cs_df, on='cancer_stage', how='inner') \
               .merge(aq_df, left_on='air_quality', right_on='air_quality', how='inner') \
               .merge(cc_df, left_on='country', right_on='country_name', how='inner')

In [12]:
result_df 

,patient_id,patient_age,patient_gender,country,diagnosis_date,end_treatment_date,cancer_stage,hypertension,asthma,cirrhosis,...,date_id_x,date_x,date_id_y,date_y,treatment_type_id,treatment,cancer_stage_id,air_quality_id,country_code,country_name
0,20,56,Male,Lithuania,2014-07,2015-08,Stage IV,1,0,0,...,1,2014-07,8,2015-08,1,Combined,s4,2,LT,Lithuania
1,30,36,Male,Italy,2014-07,2015-12,Stage IV,1,0,0,...,1,2014-07,11,2015-12,1,Combined,s4,2,IT,Italy
2,33,61,Male,Denmark,2014-10,2015-10,Stage I,1,0,0,...,2,2014-10,18,2015-10,2,Radiation,s1,2,DK,Denmark
3,46,56,Male,Sweden,2014-08,2016-03,Stage III,0,0,0,...,3,2014-08,30,2016-03,1,Combined,s3,1,SE,Sweden
4,54,51,Male,Austria,2014-08,2016-01,Stage I,1,1,0,...,3,2014-08,24,2016-01,1,Combined,s1,1,AT,Austria
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187477,3249964,65,Female,Hungary,2014-10,2016-01,Stage II,0,0,0,...,2,2014-10,24,2016-01,1,Combined,s2,4,HU,Hungary
187478,3249970,43,Male,Netherlands,2014-10,2015-07,Stage III,1,1,0,...,2,2014-10,9,2015-07,3,Surgery,s3,1,NL,Netherlands
187479,3249972,66,Female,Czech Republic,2014-10,2016-02,Stage III,0,1,0,...,2,2014-10,21,2016-02,1,Combined,s3,3,CZ,Czech Republic
187480,3249976,48,Male,Cyprus,2014-07,2015-01,Stage IV,0,0,0,...,1,2014-07,19,2015-01,3,Surgery,s4,3,CY,Cyprus


In [13]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187482 entries, 0 to 187481
Data columns (total 29 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   patient_id          187482 non-null  int64  
 1   patient_age         187482 non-null  int32  
 2   patient_gender      187482 non-null  object 
 3   country             187482 non-null  object 
 4   diagnosis_date      187482 non-null  object 
 5   end_treatment_date  187482 non-null  object 
 6   cancer_stage        187482 non-null  object 
 7   hypertension        187482 non-null  int64  
 8   asthma              187482 non-null  int64  
 9   cirrhosis           187482 non-null  int64  
 10  treatment_type      187482 non-null  object 
 11  survived            187482 non-null  int64  
 12  pm25_index          155926 non-null  float64
 13  pm10_index          169693 non-null  float64
 14  no2_index           163347 non-null  float64
 15  so2_index           157744 non-nul

In [15]:
final_df = result_df[['patient_id', 'patient_age', 'patient_gender', 'country_code', 'date_id_x', 'date_id_y', 'cancer_stage_id', 'hypertension', 'asthma', 'cirrhosis', 'treatment_type_id', 'survived', 'pm25_index', 'pm10_index', 'no2_index', 'so2_index', 'o3_index', 'air_quality_index', 'air_quality_id']]

In [16]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187482 entries, 0 to 187481
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   patient_id         187482 non-null  int64  
 1   patient_age        187482 non-null  int32  
 2   patient_gender     187482 non-null  object 
 3   country_code       187482 non-null  object 
 4   date_id_x          187482 non-null  int64  
 5   date_id_y          187482 non-null  int64  
 6   cancer_stage_id    187482 non-null  object 
 7   hypertension       187482 non-null  int64  
 8   asthma             187482 non-null  int64  
 9   cirrhosis          187482 non-null  int64  
 10  treatment_type_id  187482 non-null  int64  
 11  survived           187482 non-null  int64  
 12  pm25_index         155926 non-null  float64
 13  pm10_index         169693 non-null  float64
 14  no2_index          163347 non-null  float64
 15  so2_index          157744 non-null  float64
 16  o3

In [17]:
final_df = final_df.rename(columns={'date_id_x': 'diagnosis_date_id', 'date_id_y': 'end_treament_date_id'})

In [18]:
final_df

,patient_id,patient_age,patient_gender,country_code,diagnosis_date_id,end_treament_date_id,cancer_stage_id,hypertension,asthma,cirrhosis,treatment_type_id,survived,pm25_index,pm10_index,no2_index,so2_index,o3_index,air_quality_index,air_quality_id
0,20,56,Male,LT,1,8,s4,1,0,0,1,0,39.800000,70.292333,22.557303,0.936812,44.699282,70.292333,2
1,30,36,Male,IT,1,11,s4,1,0,0,1,1,46.968776,44.930715,22.093366,2.030453,59.572171,59.572171,2
2,33,61,Male,DK,2,18,s1,1,0,0,2,0,77.522894,62.570372,27.127633,0.851263,NaN,77.522894,2
3,46,56,Male,SE,3,30,s3,0,0,0,1,0,31.541102,31.988215,27.102650,0.610605,48.417723,48.417723,1
4,54,51,Male,AT,3,24,s1,1,1,0,1,0,29.641690,27.843433,18.575604,1.154375,45.687438,45.687438,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187477,3249964,65,Female,HU,2,24,s2,0,0,0,1,0,164.854167,80.019868,5.228226,0.760274,NaN,164.854167,4
187478,3249970,43,Male,NL,2,9,s3,1,1,0,3,0,49.662521,49.288220,45.004958,2.968721,19.011403,49.662521,1
187479,3249972,66,Female,CZ,2,21,s3,0,1,0,1,0,130.945211,71.710212,27.717802,2.848693,22.786742,130.945211,3
187480,3249976,48,Male,CY,1,19,s4,0,0,0,3,1,99.622401,102.840323,NaN,NaN,NaN,102.840323,3


In [21]:
final_df['patient_id'].duplicated().sum()

0